## Loading packages

In [ ]:
# Import required packages
import warnings
import matplotlib.pyplot as plt

import cupid_utils.generic_fxns as gfxns

# Suppress Cartopy warnings (safe to keep even if Cartopy is not used)
warnings.filterwarnings("ignore")

In [ ]:
CESM_output_dir = ""
case_name = ""
case_nickname = ""
base_case_output_dir = None
base_case_name = None
base_case_nickname = None

In [ ]:
# Want base case dir to default to control case value
if base_case_name is not None:
    if base_case_output_dir is None:
        base_case_output_dir = CESM_output_dir

### Read CESM output

In [ ]:
# Define cases with short labels

cases = {
    case_name: {
        "label": case_nickname,
        "offset": 0,  # TODO: make configurable
        "descr": "case",
        "path": CESM_output_dir,
    },
    base_case_name: {
        "label": base_case_nickname,
        "offset": 0,
        "descr": "baseline case",
        "path": base_case_output_dir,
    },
}

In [ ]:
# Compute global annual mean TS for each case
TS_global = {}
for casename, info in cases.items():
    label = info["label"]
    TS_global[label] = gfxns.compute_var_g_ann(info["path"], casename, "TS")

In [ ]:
# read ICEFRAC for Lab sea
lat1 = 53.5
lat2 = 65.3
lon1 = 360 - 63.5
lon2 = 360 - 47.5

# Compute global annual mean TS for each case
ICE_labsea = {}
for casename, info in cases.items():
    label = info["label"]
    ICE_labsea[label] = gfxns.compute_var_lat_lon_ann(
        info["path"], casename, "ICEFRAC", lat1, lat2, lon1, lon2
    )

In [ ]:
# Compute global annual mean RESTOM for each case
RESTOM_global = {}
RESTOM_global_after20 = {}  # dropping the first 20 years of the run
restom_5yr_avg = {}
for casename, info in cases.items():
    label = info["label"]

    # --- restom global ---
    RESTOM_global[label] = gfxns.compute_var_g_ann(info["path"], casename, "RESTOM")

    # --- drop first 20 years ---
    first_year = RESTOM_global[label].year.min().item()
    restom_after20 = RESTOM_global[label].where(
        RESTOM_global[label].year >= first_year + 20, drop=True
    )
    RESTOM_global_after20[label] = restom_after20

    # 5-year running mean
    restom_5yr_avg[label] = RESTOM_global[label].rolling(year=5, center=True).mean()

# 5 year running averaging but RESTOM in  legend is not running average

In [ ]:
import matplotlib.pyplot as plt
import itertools

# Set up plot
fig = plt.figure(figsize=(20, 10))
plt.subplots_adjust(hspace=0.3)
font_size_axis = 12

# Define automatic styles
colors = ["black", "red", "blue", "green", "orange"]
colors = ["black", "red", "blue", "green"]
colors = ["black", "red", "blue"]
linestyles = ["-", "--", ":", "-."]

# First subplot
ax = fig.add_subplot(2, 2, 1)
style_cycle = itertools.cycle(
    [(color, ls) for ls in linestyles for color in colors]
)  # cycle through colors, then linestyle
for casename, info in cases.items():
    label = info["label"]
    offset = info["offset"]
    ts = TS_global.get(label)
    if ts is not None:
        color, linestyle = next(style_cycle)
        ax.plot(
            ts.year + offset,
            ts,
            color=color,
            linestyle=linestyle,
            label=label,
            linewidth=2.0,
        )
ax.set_xlabel("Time (year)", fontsize=font_size_axis)
ax.set_ylabel("TS (K)", fontsize=font_size_axis)
ax.set_title("TS", fontsize=20)
ax.tick_params(axis="both", which="major", labelsize=14)
ax.grid()

# Second subplot
ax = fig.add_subplot(2, 2, 2)
style_cycle = itertools.cycle([(color, ls) for ls in linestyles for color in colors])
for casename, info in cases.items():
    label = info["label"]
    offset = info["offset"]
    ice_labsea = ICE_labsea.get(label)
    if ice_labsea is not None:
        color, linestyle = next(style_cycle)
        ax.plot(
            ice_labsea.year + offset,
            ice_labsea,
            color=color,
            linestyle=linestyle,
            label=label,
            linewidth=2.0,
        )

ax.set_xlabel("Time (year)", fontsize=font_size_axis)
ax.set_ylabel("ICEFRAC", fontsize=font_size_axis)
ax.set_title("ICEFRAC (Lab sea)", fontsize=20)
ax.tick_params(axis="both", which="major", labelsize=14)
ax.set_ylim(0, 0.5)
ax.grid()

# Third subplot
ax = fig.add_subplot(2, 2, 3)

style_cycle = itertools.cycle([(color, ls) for ls in linestyles for color in colors])
for casename, info in cases.items():
    label = info["label"]
    offset = info["offset"]
    descr = info["descr"]
    restom_5yr = restom_5yr_avg.get(label)
    restom = RESTOM_global.get(label)
    if restom_5yr is not None:
        color, linestyle = next(style_cycle)
        ax.plot(
            restom_5yr.year + offset,
            restom_5yr,
            color=color,
            linestyle=linestyle,
            label=f"{label}: {restom.mean().item():.2f} W/m2  ({descr})",
            linewidth=2.0,
        )

ax.set_xlabel("Time (year)", fontsize=font_size_axis)
ax.set_ylabel("RESTOM (W/m2)", fontsize=font_size_axis)
ax.set_title("RESTOM (5-year running avg)", fontsize=20)
ax.tick_params(axis="both", which="major", labelsize=14)

# Adding legend with title
legend = ax.legend(
    loc="center left", bbox_to_anchor=(1.3, 0.5), fontsize="16", title="RESTOM"
)
legend.get_title().set_fontsize("18")  # Setting the font size of the legend title

ax.grid()

plt.show()